In [57]:
import requests
import json
import random 

URL = "http://localhost:"
ports = range(10000,10006)
def random_url():
    return (URL + str(random.choice(ports))+"/")

def address(addr):
    return str(addr).rjust(16, '0')


### View initial state

In [58]:
initial_utxo = requests.get(random_url() + "utxos/" + address(0)).content.decode('utf-8')
print(f"INITIAL UTXO\n {initial_utxo}")

INITIAL UTXO
 {
  "utxos": [{
    "txId": "0x00000000001",
    "utxoId": "00000000-0000-0000-0000-000000000000",
    "addr": "0000000000000000",
    "coins": "10000"
  }]
}


### Send money between addresses handled by different shards


In [59]:
# from 0 to 1
requests.post(random_url() + "sendCoins", params={"to": address(1), "from": address(0), "coins": "100"})
# from 1 to 2
requests.post(random_url() + "sendCoins", params={"to": address(2), "from": address(1), "coins": "90"})
# from 2 to 3
requests.post(random_url() + "sendCoins", params={"to": address(3), "from": address(2), "coins": "80"})
# from 3 to 4
requests.post(random_url() + "sendCoins", params={"to": address(4), "from": address(3), "coins": "70"})
# from 4 to 1
requests.post(random_url() + "sendCoins", params={"to": address(1), "from": address(4), "coins": "60"})

<Response [200]>

### Get UTxOs after send money series

In [60]:
for addr in range(1, 5):
	utxos = requests.get(random_url() + "utxos/" + address(addr)).content.decode('utf-8')
	print(f"\n{addr} UTxOs\n\n{utxos}")


1 UTxOs

{
  "utxos": [{
    "txId": "ae8af4c7-4855-46e9-8790-7a8dcc73298c",
    "utxoId": "c30200d4-3508-31f8-b761-57d6b794a342",
    "addr": "0000000000000001",
    "coins": "10"
  }]
}

2 UTxOs

{
  "utxos": [{
    "txId": "dc51d422-c315-4589-aca0-9c7adaf1d97c",
    "utxoId": "8270c62e-2b4d-3936-b1bc-068b7e9edd9c",
    "addr": "0000000000000002",
    "coins": "10"
  }]
}

3 UTxOs

{
  "utxos": [{
    "txId": "67fb3cc3-efd1-4462-aec1-816c77024943",
    "utxoId": "2f041397-2987-3471-8251-9dcc5f636c5f",
    "addr": "0000000000000003",
    "coins": "10"
  }]
}

4 UTxOs

{
}


### Get ledger history for address 0

In [61]:
ledger = requests.get(random_url() + "ledger/" + address(0)).content.decode('utf-8')
print(f"Ledger for 0 \n{ledger}")

Ledger for 0 
This is the history for address 0000000000000000:
{
  "txs": [{
    "txId": "fd1a7a7a-58bb-4690-b300-f25cacffd2b6",
    "inputs": [{
      "txId": "0x00000000001",
      "utxoId": "00000000-0000-0000-0000-000000000000",
      "addr": "0000000000000000",
      "coins": "10000"
    }],
    "outputs": [{
      "addr": "0000000000000001",
      "coins": "100"
    }, {
      "addr": "0000000000000000",
      "coins": "9900"
    }],
    "timestamp": "2022-01-26T12:08:17.601Z"
  }]
}


### Get entire ledger history

In [62]:
ledger = requests.get(random_url() + "ledger/").content.decode('utf-8')
print(f"{ledger}")

This is the entire ledger history at the moment:
{
  "txs": [{
    "txId": "67fb3cc3-efd1-4462-aec1-816c77024943",
    "inputs": [{
      "txId": "dc51d422-c315-4589-aca0-9c7adaf1d97c",
      "utxoId": "77007dea-d712-38fe-9acf-10dc4ae5cd19",
      "addr": "0000000000000003",
      "coins": "80"
    }],
    "outputs": [{
      "addr": "0000000000000004",
      "coins": "70"
    }, {
      "addr": "0000000000000003",
      "coins": "10"
    }],
    "timestamp": "2022-01-26T12:08:57.761Z"
  }, {
    "txId": "dc51d422-c315-4589-aca0-9c7adaf1d97c",
    "inputs": [{
      "txId": "ae8af4c7-4855-46e9-8790-7a8dcc73298c",
      "utxoId": "1e7c50b4-5f94-3584-a64e-380e56b68545",
      "addr": "0000000000000002",
      "coins": "90"
    }],
    "outputs": [{
      "addr": "0000000000000003",
      "coins": "80"
    }, {
      "addr": "0000000000000002",
      "coins": "10"
    }],
    "timestamp": "2022-01-26T12:08:51.159Z"
  }, {
    "txId": "ae8af4c7-4855-46e9-8790-7a8dcc73298c",
    "inputs": [

### Make a transaction

Transfer half of the coins for address 1 to address 0

In [66]:
utxos_1 = json.loads(requests.get(random_url() + "utxos/" + address(0)).content.decode('utf-8'))
coins = sum([int(utxo['coins']) for utxo in utxos_1['utxos']]) 
half_coins = coins //2
output1 = {"addr": address(1), "coins": str(half_coins)}
output2 = {"addr": address(0), "coins": str(coins-half_coins)}
new_tx = {}
new_tx['inputs'] = utxos_1['utxos']
new_tx['outputs'] = output1,output2
new_tx=json.dumps(new_tx).encode('utf-8')
res = requests.post(random_url() + "submitTx", data=new_tx, headers={'Content-Type': 'application/json'})
print(res.content.decode('utf-8'))

Submitted successfully with ID 16cdd2ed-203a-40c6-8e65-d12a1c5b055d 


### Print updated UTxOs for 0 and 1 after the transaction

In [65]:
for addr in range(0, 2):
	utxos = requests.get(random_url() + "utxos/" + address(addr)).content.decode('utf-8')
	print(f"\n{addr} UTxOs\n\n{utxos}")


0 UTxOs

{
  "utxos": [{
    "txId": "dc895ee3-ee31-4007-9781-9ced9cb6790b",
    "utxoId": "2702ec58-2fd0-3be4-908a-b57e1585b7d2",
    "addr": "0000000000000000",
    "coins": "4950"
  }]
}

1 UTxOs

{
  "utxos": [{
    "txId": "ae8af4c7-4855-46e9-8790-7a8dcc73298c",
    "utxoId": "c30200d4-3508-31f8-b761-57d6b794a342",
    "addr": "0000000000000001",
    "coins": "10"
  }]
}
